In [ ]:
# 1
import pandas as pd
import numpy as np
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import matplotlib.dates
import urllib.request
import time
import datetime
from google.colab import files
from google.colab import data_table

In [ ]:
# 2
def dateToTimestamp(d):
  return time.mktime(datetime.datetime.strptime(d,"%d/%m/%Y").timetuple())
  
def dateToDatetime(d,mask):
  return datetime.datetime.strptime(d,mask)

In [ ]:
# 3
# arquivo CSV de relatório de faturamento por linha/veiculo/viagem gerado pelo ecitop
uploaded = files.upload()
csv_file = []
for fn in uploaded.keys():
  print('User uploaded file "{csv_file}" with length {length} bytes'.format(csv_file=fn, length=len(uploaded[fn])))
  csv_file.append(fn)

Saving abril2023_part3.csv to abril2023_part3.csv
Saving abril2023_part2.csv to abril2023_part2.csv
Saving abril2023_part1.csv to abril2023_part1.csv
User uploaded file "abril2023_part3.csv" with length 162016222 bytes
User uploaded file "abril2023_part2.csv" with length 242629991 bytes
User uploaded file "abril2023_part1.csv" with length 188738758 bytes


In [ ]:
# 4
valor_passagem = 2.00;  # para filtro por valor de passagem e não por horário
byHour = True           # se True: filtra pelo array de horários; se False: filtra por valor da passagem
totaliza = True         # se True: totaliza todas as operadoras; se False: totaliza a operadora na variável op
picoEntrepico = False    # se True: filtra pelo array de horários considerando os horários de pico 
op = 19                 # apenas seleciona por operadora se totaliza=False
mask = "%d/%m/%Y %H:%M:%S"
gr = ["Data Coleta",'Ano',"Código Operadora"]
if byHour:
   gr = ["Data Coleta","Hora","Código Operadora"]
operadoras = {
              19: "Rosa",
              20: "São João",
              21: "Via Feira"
             } # operadoras: 19 Rosa, 20 São João, 21 Via Feira
horariosEntrepico = {
    0:[9,10,13,14,20,21,22,23],
    1:[9,10,13,14,20,21,22,23],
    2:[9,10,13,14,20,21,22,23],
    3:[9,10,13,14,20,21,22,23],
    4:[9,10,13,14,20,21,22,23],
    5:[14,15,16,17,18,19,20,21,22,23],
    6:[9,10,13,14,20,21,22,23],
}

horariosPico = {
    0:[4,5,6,7,8,11,12,15,16,17,18,19],
    1:[4,5,6,7,8,11,12,15,16,17,18,19],
    2:[4,5,6,7,8,11,12,15,16,17,18,19],
    3:[4,5,6,7,8,11,12,15,16,17,18,19],
    4:[4,5,6,7,8,11,12,15,16,17,18,19]
}

horariosGeral = {
    0:[4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23],
    1:[4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23],
    2:[4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23],
    3:[4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23],
    4:[4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23],
    5:[4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23],
    6:[4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23]
}

horariosFaixa = {
    "04h às 8:59h":[4,5,6,7,8],
    "12h às 12:59h":[12],
    "15h às 19:59h":[15,16,17,18,19],
    "09h às 10:59h":[9,10],
    "13h às 14:59h":[13,14],
    "20h às 23:59h":[20,21,22,23]
}


In [ ]:
#5
df = pd.DataFrame()
for url in csv_file:
    print("Reading...",url)
    r = pd.read_csv(url, 
                    sep=';', 
                    index_col=False,
                    header=0,
                    encoding='utf-8', 
                    decimal=",", 
                    dtype={
                        'Quebra Composta': str,
                        'Numero Veículo': str, 
                        'Código Externo Linha': str, 
                        'Cartão Cobrador': str, 
                        'Cartão Motorista': str, 
                        'VT': int,
                        'Valor VT': float,
                        'Passagens': int,
                        'Valor Passagens': float
                        }
                    )
    df = df.append(r)

Reading... abril2023_part3.csv


<ipython-input-5-7948ba3d5386>:5: DtypeWarning: Columns (23,28,31) have mixed types. Specify dtype option on import or set low_memory=False.
  r = pd.read_csv(url,
<ipython-input-5-7948ba3d5386>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(r)


Reading... abril2023_part2.csv


<ipython-input-5-7948ba3d5386>:5: DtypeWarning: Columns (23,28,31) have mixed types. Specify dtype option on import or set low_memory=False.
  r = pd.read_csv(url,
<ipython-input-5-7948ba3d5386>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(r)


Reading... abril2023_part1.csv


<ipython-input-5-7948ba3d5386>:5: DtypeWarning: Columns (23,28,31) have mixed types. Specify dtype option on import or set low_memory=False.
  r = pd.read_csv(url,
<ipython-input-5-7948ba3d5386>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(r)


In [ ]:
df.rename(columns = {
    'Numero CartÃ£o':'Numero Cartao',
    'Data Hora UtilizaÃ§Ã£o':'Data Hora Utilização',
    'CÃ³digo Operadora':'Código Operadora'
    }, inplace = True)

In [ ]:
display(df.head(5))

,Numero Cartão,Data Hora Utilização,Código Operadora,Data Coleta,Código Interno Linha,Código Veículo,Código Equipamento,Viagem,Quebra Composta,Nome Operadora,...,Município,Total Eletrônico,Valor Total Eletrônico,Passagens,Passagens Integração,Valor Passagens,Valor Passagens Integração,Bilhete Unitário,Valor Bilhete Unitário,Unnamed: 53
0,03620000060622,2023-04-21 06:01:06.0,19,21/04/2023,107,297,75,16339896,TN0141141,EMPRESA DE ONIBUS ROSA LTDA,...,FEIRA DE SANTANA,1,2.07,1,0,2.07,0,0,0,NaN
1,06700000010670,2023-04-21 06:04:21.0,19,21/04/2023,107,297,75,16339896,TN0141141,EMPRESA DE ONIBUS ROSA LTDA,...,FEIRA DE SANTANA,1,2.07,1,0,2.07,0,0,0,NaN
2,03620000156976,2023-04-21 06:08:47.0,19,21/04/2023,107,297,75,16339896,TN0141141,EMPRESA DE ONIBUS ROSA LTDA,...,FEIRA DE SANTANA,1,2.07,1,0,2.07,0,0,0,NaN
3,06850000114640,2023-04-21 06:11:22.0,19,21/04/2023,107,297,75,16339896,TN0141141,EMPRESA DE ONIBUS ROSA LTDA,...,FEIRA DE SANTANA,1,3.78,0,0,0.00,0,0,0,NaN
4,Inteira,2023-04-21 06:16:44.0,19,21/04/2023,107,297,75,16339896,TN0141141,EMPRESA DE ONIBUS ROSA LTDA,...,FEIRA DE SANTANA,0,0.00,0,0,0.00,0,0,0,NaN


In [ ]:
display(df.nunique())

In [ ]:
display(df.dtypes)

In [ ]:
# 6
df['Data Coleta'] = pd.to_datetime(df['Data Hora Utilização'], errors='coerce')
df['Data Coleta'] = df['Data Coleta'].dt.strftime("%d/%m/%Y")
df['Data Hora Utilização'] = pd.to_datetime(df['Data Hora Utilização'], errors='coerce')
df['Hora'] = df['Data Hora Utilização'].dt.hour

dataInicio = min(df['Data Hora Utilização'])
dataFinal   = max(df['Data Hora Utilização'])
print(dataInicio)
print(dataFinal)
# dataInicio = '01/03/2019 00:00:00'
# dataFinal = '31/03/2022 23:59:59'
# print(dateToDatetime(dataInicio,mask))
# print(dateToDatetime(dataFinal,mask))

2023-04-01 04:24:36
2023-04-30 21:16:25


In [ ]:
# 7 ou
# totalização por operadora e valor da passagem - RODAR ESTE
# ATUALIZADO: válido apenas para períodos de vigência do decreto
demanda = pd.DataFrame()
def demandaVlrPassagem(op):
    rows = df.loc[
                    (
                      (df['Data Hora Utilização'] >= dataInicio) & 
                      (df['Data Hora Utilização'] <= dataFinal) &
                      (
                          (df['Valor VT'] == valor_passagem) |
                          (df['Valor Passagens'] == valor_passagem)
                      )
                      &
                      ((df['Código Operadora'] == op) if totaliza == False else (df['Código Operadora'] == 19) | (df['Código Operadora'] == 20) | (df['Código Operadora'] == 21))
                    )
                    ].groupby(gr)[["VT","Valor VT","Passagens","Valor Passagens"]].apply(sum)
    return rows

demanda = demandaVlrPassagem(op)

demanda_final = demanda.groupby(gr)[["VT","Valor VT","Passagens","Valor Passagens"]].apply(sum)
demanda_final["Total"] = demanda_final["VT"]+demanda_final["Passagens"]
demanda_final["Receita Total"] = demanda_final["Valor VT"]+demanda_final["Valor Passagens"]

In [ ]:
# 8 ou
# totalização por operadora e horário
# UPDATED: a rotina leva em consideração os horários de pico e entrepico criando faixas
# esta rotina não leva o dia da semana em consideração
demanda = pd.DataFrame()
def demandaHorario(d,h,op):
    rows = df.loc[
                    (
                      (df['Data Hora Utilização'] >= dataInicio) & 
                      (df['Data Hora Utilização'] <= dataFinal) &
                      (
                          (df['Data Hora Utilização'].dt.hour == h) &
                          (df['Data Hora Utilização'].dt.minute <= 59) &
                          (df['Data Hora Utilização'].dt.second <= 59)
                      )
                      &
                      ((df['Código Operadora'] == op) if totaliza == False else (df['Código Operadora'] == 19) | (df['Código Operadora'] == 20) | (df['Código Operadora'] == 21))
                    )
                    ].groupby(gr)[["VT","Valor VT","Passagens","Valor Passagens"]].apply(sum)
    return rows

for d in horariosFaixa:
    for h in horariosFaixa[d]:
            demanda = demanda.append(demandaHorario(d,h,op), ignore_index=False)
demanda_final = demanda.groupby(gr)[["VT","Valor VT","Passagens","Valor Passagens"]].apply(sum)
demanda_final["Total"] = demanda_final["VT"]+demanda_final["Passagens"]
demanda_final["Receita Total"] = demanda_final["Valor VT"]+demanda_final["Valor Passagens"]

In [ ]:
# printa demanda_final se for necessário (se o resultado do excel para os totais for diferente, observar a duplicação de linhas)
display(demanda_final)
print("\n")
print("TOTAIS:")
print(demanda_final[["VT","Valor VT","Passagens","Valor Passagens"]].apply(sum))
print(demanda_final.sum(axis=0))


VT  Valor VT  Passagens  \
Data Coleta Hora Código Operadora                               
01/04/2023  9.0  19                410.0     820.0      199.0   
                 20                359.0     718.0      151.0   
                 21                 35.0      70.0       23.0   
            10.0 19                411.0     822.0      204.0   
                 20                380.0     760.0      162.0   
...                                  ...       ...        ...   
30/04/2023  20.0 19                 67.0     134.0        0.0   
                 20                 54.0     108.0        0.0   
                 21                  3.0       6.0        0.0   
            21.0 19                 16.0      32.0        0.0   
                 20                  4.0       8.0        0.0   

                                   Valor Passagens  Total  Receita Total  
Data Coleta Hora Código Operadora                                         
01/04/2023  9.0  19                          398.0  609.0         1218.0  
                 20                          302.0  510.0         1020.0  
                 21                           46.0   58.0          116.0  
            10.0 19                          408.0  615.0         1230.0  
                 20                          324.0  542.0         1084.0  
...                                            ...    ...            ...  
30/04/2023  20.0 19                            0.0   67.0          134.0  
                 20                            0.0   54.0          108.0  
                 21                            0.0    3.0            6.0  
            21.0 19                            0.0   16.0           32.0  
                 20                            0.0    4.0            8.0  

[845 rows x 6 columns]



TOTAIS:
VT                 170200.0
Valor VT           340400.0
Passagens           74697.0
Valor Passagens    149394.0
dtype: float64
VT                 170200.0
Valor VT           340400.0
Passagens           74697.0
Valor Passagens    149394.0
Total              244897.0
Receita Total      489794.0
dtype: float64


In [ ]:
# salvar dados em arquivo CSV
demanda_final.to_csv('estudo_relfatporlinhaveiculoviagem.csv')